## MNIST

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import sys
sys.path.append("..")

%matplotlib inline

import numpy as np

from sklearn.metrics import classification_report

import keras
from keras import metrics
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.layers import Dense, Flatten
from keras.models import Model, load_model
import keras.backend as k
from matplotlib import pyplot as plt
from IPython.display import clear_output

from art.config import ART_DATA_PATH
from art.estimators.classification import KerasClassifier
from art.utils import to_categorical, load_dataset, get_file

import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

from deepfool import DeepFool

In [4]:
(x_train, y_train), (x_test, y_test), min_, max_ = load_dataset('mnist')

path = get_file('mnist_cnn_original.h5', extract=False, path=ART_DATA_PATH,
                url='https://www.dropbox.com/s/p2nyzne9chcerid/mnist_cnn_original.h5?dl=1')

METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]

classifier_model = load_model(path)
classifier = KerasClassifier(clip_values=(min_, max_), model=classifier_model, use_logits=True)
classifier_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=METRICS)
history = classifier.fit(x_train, y_train, nb_epochs=1, batch_size=128, verbose=1)

Epoch 1/1
468/468 [==============================] - 15s 32ms/step - loss: 4.7499e-04 - tp: 59898.0000 - fp: 6.0000 - tn: 539130.0000 - fn: 6.0000 - categorical_accuracy: 0.9999 - precision: 0.9999 - recall: 0.9999 - auc: 1.0000


In [5]:
classifier_model.summary()
pred = classifier.predict(x_test)
x_test_pred = np.argmax(pred, axis=1)
nb_correct_pred = np.sum(x_test_pred == np.argmax(y_test, axis=1))
accuracy = np.mean(np.argmax(pred, axis=1) == np.argmax(y_test, axis=1))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               204928    
_________________________________________________________________
dense_2 (Dense)              (None, 10)               

In [6]:
print(f"---Original test images---:")
print("Correctly classified: {}".format(nb_correct_pred))
print("Accuracy on test samples: %f" % accuracy)

---Original test images---:
Correctly classified: 9917
Accuracy on test samples: 0.991700


In [ ]:
attacker = DeepFool(classifier)
x_test_adv = attacker.generate(x_test)

W1203 01:30:55.698018 4465061312 deepfool.py:80] Targeted model should output logits, not probabilities for predictions.
DeepFool:  67%|██████▋   | 6720/10000 [39:30<17:33,  3.11it/s]  

In [ ]:
preds = np.argmax(classifier.predict(x_test_adv), axis=1)
acc = np.sum(preds == np.argmax(y_test, axis=1)) / y_test.shape[0]

print(np.sum(preds == np.argmax(y_test, axis=1)))

## CFAIR-10

In [2]:
import logging

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout

In [3]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
formatter = logging.Formatter("[%(levelname)s] %(message)s")
handler.setFormatter(formatter)
logger.addHandler(handler)

In [4]:
(x_train, y_train), (x_test, y_test), min_, max_ = load_dataset(str("cifar10"))
x_train, y_train = x_train[:5000], y_train[:5000]
x_test, y_test = x_test[:500], y_test[:500]

In [5]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]

model = Sequential()
model.add(Conv2D(32, (3, 3), padding="same", input_shape=x_train.shape[1:]))
model.add(Activation("relu"))
model.add(Conv2D(32, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=METRICS)

In [6]:
classifier = KerasClassifier(model=model, clip_values=(min_, max_))
classifier.fit(x_train, y_train, nb_epochs=10, batch_size=128, verbose=1)

[INFO] Inferred 17 hidden layers on Keras classifier.


Epoch 1/10
39/39 [==============================] - 6s 158ms/step - batch: 19.0000 - size: 128.0000 - loss: 2.1712 - tp: 30.0000 - fp: 46.0000 - tn: 44882.0000 - fn: 4962.0000 - categorical_accuracy: 0.1871 - precision: 0.3947 - recall: 0.0060 - auc: 0.6478
Epoch 2/10
39/39 [==============================] - 6s 157ms/step - batch: 19.0000 - size: 128.0000 - loss: 1.8452 - tp: 310.0000 - fp: 248.0000 - tn: 44680.0000 - fn: 4682.0000 - categorical_accuracy: 0.3277 - precision: 0.5556 - recall: 0.0621 - auc: 0.7903
Epoch 3/10
39/39 [==============================] - 6s 157ms/step - batch: 19.0000 - size: 128.0000 - loss: 1.6738 - tp: 649.0000 - fp: 425.0000 - tn: 44503.0000 - fn: 4343.0000 - categorical_accuracy: 0.3888 - precision: 0.6043 - recall: 0.1300 - auc: 0.8335
Epoch 4/10
39/39 [==============================] - 6s 157ms/step - batch: 19.0000 - size: 128.0000 - loss: 1.5496 - tp: 913.0000 - fp: 501.0000 - tn: 44427.0000 - fn: 4079.0000 - categorical_accuracy: 0.4365 - precision: 

In [7]:
logger.info("Create DeepFool attack")
adv_crafter = DeepFool(classifier)
#logger.info("Craft attack on training examples")
#x_train_adv = adv_crafter.generate(x_train)
logger.info("Craft attack test examples")
x_test_adv = adv_crafter.generate(x_test)

[INFO] Create DeepFool attack
[INFO] Craft attack test examples
[WARNING] Targeted model should output logits, not probabilities for predictions.
DeepFool: 100%|██████████| 500/500 [01:11<00:00,  6.99it/s]
[INFO] DeepFool attack success rate: 88.60%


In [8]:
preds = np.argmax(classifier.predict(x_test_adv), axis=1)
acc = np.sum(preds == np.argmax(y_test, axis=1)) / y_test.shape[0]
logger.info("Classifier before adversarial training")
logger.info("Accuracy on adversarial samples: %.2f%%", (acc * 100))
classifier.model.evaluate(x_test_adv, y_test, verbose=1)

[INFO] Classifier before adversarial training
[INFO] Accuracy on adversarial samples: 22.40%


[1.7357193355560303,
 16.0,
 74.0,
 4426.0,
 484.0,
 0.224,
 0.17777778,
 0.032,
 0.84140915]

## Extension

In [7]:
from algorithms.fast_deepfool import FastDeepFool

In [8]:
logger.info("Create DeepFool attack")
adv_crafter = FastDeepFool(classifier)
logger.info("Craft attack on test examples")
x_test_adv = adv_crafter.generate(x_test)

[INFO] Create DeepFool attack
[INFO] Craft attack on test examples
[WARNING] Targeted model should output logits, not probabilities for predictions.
DeepFool: 100%|██████████| 500/500 [01:33<00:00,  5.34it/s]
[INFO] DeepFool attack success rate: 99.80%
